In [4]:
from langchain.vectorstores import Chroma
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader , PyPDFLoader
import os

In [9]:
curr_dir = os.getcwd()
combined_docs = []
for path in os.listdir(curr_dir):
    file_fmt = os.path.splitext(path)[1]
    if file_fmt==".pdf":
        print("Pdf is loading..........")
        loader = PyPDFLoader(path)
        combined_docs.extend(loader.load())
        print("The PDF loaded successfully!")
    elif file_fmt==".txt":
        print("The txt file is loading..........")
        loader = TextLoader(path)
        combined_docs.extend(loader.load())
        print("The TXT loaded successfully!")
    else:
        print("File format is not supported")


File format is not supported
Pdf is loading..........
The PDF loaded successfully!


In [11]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000 , chunk_overlap=200)
splitted_docs = text_splitter.split_documents(combined_docs)

embeddings_model = OllamaEmbeddings(model="nomic-embed-text:latest")
db = Chroma.from_documents(splitted_docs , embeddings_model)
retriever = db.as_retriever()

In [12]:
from dotenv import load_dotenv

load_dotenv()

os.environ["GOOGLE_API_KEY"] = os.getenv("GEMINI_API_KEY")

from langchain_google_genai.llms import GoogleGenerativeAI

llm = GoogleGenerativeAI(model="gemini-2.0-flash")



In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import  SequentialChain 
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain

refining_prompt = PromptTemplate(
    template="""
Given the question: {input},
and the retrieved context: {context},
refine the question for further retrieval.
""", input_variables=["question" , "context"]
)
final_prompt = PromptTemplate(
    template="""
Use the refined question: {input} 
to retrieve more context and answer it. 
Context: {context}
""",input_variables=["refined_question" , "context"]
)

refining_1 = create_stuff_documents_chain(llm = llm , prompt=refining_prompt )
refining_2 = create_retrieval_chain(retriever=retriever , combine_docs_chain=refining_1 )


final_1 = create_stuff_documents_chain(llm=llm , prompt=refining_prompt)
final_2 = create_retrieval_chain(retriever=retriever , combine_docs_chain=final_1)


In [33]:
def query(question):
    refined_question = refining_2.invoke({"input":question})["answer"]
    output = final_2.invoke({"input":refined_question})["answer"]

    return output

In [34]:
query("How does qr based diffusion work")

'Okay, based on the original question and the retrieved context, let\'s refine the question to target more specific aspects of DiffQRCoder.  Here are a few options, depending on what we want to learn more about:\n\n**Option 1 (Focus on SRPG):**\n\n*   **Refined Question:**  How does Scanning Robust Perceptual Guidance (SRPG) work within DiffQRCoder\'s diffusion process to ensure scannability?  Specifically, what criteria or mechanisms are used within SRPG to guide the diffusion model toward generating scannable QR codes, and how does it maintain aesthetics?\n\n**Reasoning:** This option digs deeper into the core innovation of DiffQRCoder - the SRPG. It\'s more specific than the original question and aims to understand the *how* behind its effectiveness.\n\n**Option 2 (Focus on the Two-Stage Process):**\n\n*   **Refined Question:**  The context mentions a "two-stage iterative refinement framework." What are the two stages in DiffQRCoder\'s process, and what is the role of SRPG and Scann

'Okay, based on the original question and the retrieved context, let\'s refine the question to target more specific aspects of DiffQRCoder.  Here are a few options, depending on what we want to learn more about:\n\n**Option 1 (Focus on SRPG):**\n\n*   **Refined Question:**  How does Scanning Robust Perceptual Guidance (SRPG) work within DiffQRCoder\'s diffusion process to ensure scannability?  Specifically, what criteria or mechanisms are used within SRPG to guide the diffusion model toward generating scannable QR codes, and how does it maintain aesthetics?\n\n**Reasoning:** This option digs deeper into the core innovation of DiffQRCoder - the SRPG. It\'s more specific than the original question and aims to understand the *how* behind its effectiveness.\n\n**Option 2 (Focus on the Two-Stage Process):**\n\n*   **Refined Question:**  The context mentions a "two-stage iterative refinement framework." What are the two stages in DiffQRCoder\'s process, and what is the role of SRPG and Scanning Robust Manifold Projected Gradient Descent (SR-MPGD) in each stage?  Why is a two-stage approach used?\n\n**Reasoning:** This explores the overall architecture of DiffQRCoder and the interplay between its key components. Understanding the two-stage nature provides a better grasp of the system as a whole.\n\n**Option 3 (Focus on SR-MPGD):**\n\n*   **Refined Question:** How does Scanning Robust Manifold Projected Gradient Descent (SR-MPGD) work as a post-processing technique in DiffQRCoder? How does it perform "latent space optimization" to enhance scanning robustness, and what are the key parameters (e.g., step size, LPIPS λ) that control its behavior?\n\n**Reasoning:** This focuses on the post-processing step, SR-MPGD.  It\'s important to understand how this technique can bring the Scanning Success Rate (SSR) up to 100%.\n\n**Option 4 (Focus on Implementation Details):**\n\n*   **Refined Question:** What Diffusion Model and ControlNet architectures are used in DiffQRCoder? Are there any specific modifications or configurations applied to these architectures to optimize them for QR code generation?\n\n**Reasoning:** This delves into the more technical aspects of the implementation. It would be useful if you\'re trying to replicate or build upon DiffQRCoder.\n\n**Which option is best depends on your goals.** If you want to understand the core innovation for scannability, choose Option 1. If you want to understand the overall system architecture, choose Option 2. If you\'re interested in the post-processing step, choose Option 3. And if you want to know about the specific models used, choose Option 4.\n\nI recommend starting with Option 1 or 2, as they provide a more fundamental understanding of DiffQRCoder. Then, you can delve into the details of SR-MPGD (Option 3) or the implementation details (Option 4) if needed.\n'